In [1]:
import deep_image_matching as dim

params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "matching_lowres",
    "quality": "medium",
    "tiling": "none",
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}
config = dim.Config(params)
imgs_dir = config.general["image_dir"]
output_dir = config.general["output_dir"]

/home/francesco/dev/deep-image-matching/src/deep_image_matching/thirdparty/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
xFormers not available
xFormers not available


Deep Image Matching loaded in 5.091 seconds.


Run feature extraction and matching


In [2]:
matcher = dim.ImageMatcher(config)
feature_path, match_path = matcher.run()

Loaded SuperPoint model
2025-07-04 18:41:41 | [INFO    ] Running image matching with the following configuration:
2025-07-04 18:41:41 | [INFO    ]   Image folder: assets/example_cyprus/images
2025-07-04 18:41:41 | [INFO    ]   Output folder: assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_medium
2025-07-04 18:41:41 | [INFO    ]   Number of images: 10
2025-07-04 18:41:41 | [INFO    ]   Matching strategy: matching_lowres
2025-07-04 18:41:41 | [INFO    ]   Image quality: MEDIUM
2025-07-04 18:41:41 | [INFO    ]   Tile selection: NONE
2025-07-04 18:41:41 | [INFO    ]   Feature extraction method: superpoint
2025-07-04 18:41:41 | [INFO    ]   Matching method: lightglue
2025-07-04 18:41:41 | [INFO    ]   Geometric verification: PYDEGENSAC
2025-07-04 18:41:41 | [INFO    ]   CUDA available: False
2025-07-04 18:41:41 | [INFO    ] Low resolution matching, generating pairs ..
Loaded SuperPoint model
2025-07-04 18:41:41 | [INFO    ] Extracting features from downsampled ima

100%|██████████| 10/10 [00:14<00:00,  1.48s/it]

2025-07-04 18:41:56 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:43<00:00,  1.02it/s]

2025-07-04 18:42:40 | [INFO    ] Found 28 pairs.
2025-07-04 18:42:40 | [INFO    ] Extracting features with superpoint...
2025-07-04 18:42:40 | [INFO    ] superpoint configuration: 


{'keypoint_threshold': 0.0005,
 'max_keypoints': 2048,
 'name': 'superpoint',
 'nms_radius': 3}


100%|██████████| 10/10 [00:12<00:00,  1.27s/it]

2025-07-04 18:42:53 | [INFO    ] Features extracted!
2025-07-04 18:42:53 | [INFO    ] Matching features with lightglue...
2025-07-04 18:42:53 | [INFO    ] lightglue configuration: 
{'depth_confidence': 0.95,
 'filter_threshold': 0.1,
 'flash': True,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'width_confidence': 0.99}
2025-07-04 18:42:53 | [INFO    ] Matching features...
2025-07-04 18:42:53 | [INFO    ] 



100%|██████████| 28/28 [01:12<00:00,  2.59s/it]

2025-07-04 18:44:05 | [INFO    ] [Timer] | [matching] generate_pairs=60.908, extract_features=12.701, Match pair=1.677, Total execution=146.011
2025-07-04 18:44:05 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


Export matches to COLMAP format


In [ ]:
# First remove existing database if it exists
database_path = output_dir / "database.db"
if database_path.exists():
    database_path.unlink()

# Export to COLMAP format
dim.io.export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_config_path=config.general["camera_options"],
)

2025-07-04 18:44:05 | [WARNING ] Was not possible to load the first image to initialize cam0
2025-07-04 18:44:05 | [WARNING ] Was not possible to load the first image to initialize cam1


  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'tolist'

Run reconstruction with pycolmap


In [ ]:
# Optional - You can specify some reconstruction configuration
# reconst_opts = (
#     {
#         "ba_refine_focal_length": True,
#         "ba_refine_principal_point": False,
#         "ba_refine_extra_params": False,
#     },
# )
reconst_opts = {}
model = dim.reconstruction.incremental_reconstruction(
    database_path=output_dir / "database.db",
    image_dir=imgs_dir,
    sfm_dir=output_dir / "reconstruction",
    reconstruction_options=reconst_opts,
    refine_intrinsics=True,
    ignore_two_view_tracks=True,
    filter_min_tri_angle=None,
    export_ply=True,
    export_text=True,
    export_bundler=False,
)


2025-07-02 14:33:46 | [INFO    ] Running 3D reconstruction...


I20250702 14:33:46.073336 140420164540224 incremental_pipeline.cc:253] Loading database
I20250702 14:33:46.085152 140420164540224 database_cache.cc:66] Loading rigs...
I20250702 14:33:46.085198 140420164540224 database_cache.cc:76]  0 in 0.000s
I20250702 14:33:46.085208 140420164540224 database_cache.cc:84] Loading cameras...
I20250702 14:33:46.086809 140420164540224 database_cache.cc:102]  1 in 0.002s
I20250702 14:33:46.086836 140420164540224 database_cache.cc:110] Loading frames...
I20250702 14:33:46.086866 140420164540224 database_cache.cc:127]  0 in 0.000s
I20250702 14:33:46.086873 140420164540224 database_cache.cc:135] Loading matches...
I20250702 14:33:46.087203 140420164540224 database_cache.cc:140]  28 in 0.000s
I20250702 14:33:46.087211 140420164540224 database_cache.cc:156] Loading images...
I20250702 14:33:46.088835 140420164540224 database_cache.cc:241]  10 in 0.002s (connected 8)
I20250702 14:33:46.088877 140420164540224 database_cache.cc:252] Building correspondence graph

2025-07-02 14:33:56 | [INFO    ] Reconstructed 1 model(s).
2025-07-02 14:33:56 | [INFO    ] Largest model is #0 with 8 images.
2025-07-02 14:33:56 | [INFO    ] Exporting model...


I20250702 14:33:56.518462 140420164540224 incremental_pipeline.cc:581] Keeping successful reconstruction
I20250702 14:33:56.519494 140420164540224 timer.cc:91] Elapsed time: 0.174 [minutes]


AttributeError: 'pycolmap._core.Reconstruction' object has no attribute 'export_bundler'